In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qt
import scipy as sp

In [17]:
sx = qt.operators.sigmax()
sy = qt.operators.sigmay()
sz = qt.operators.sigmaz()

In [54]:
def hamiltonian_ising(N,omega,delta,V):
  sx = qt.operators.sigmax()
  sy = qt.operators.sigmay()
  sz = qt.operators.sigmaz()
  I = qt.operators.identity(2)

  ising_ham = qt.tensor([I*0]*N)
  for i in range(N):
    hz = qt.tensor([sz if j==i else I for j in range(N)])
    hx = qt.tensor([sx if j==i else I for j in range(N)])
    ising_ham += -1*delta*hz+omega*hx
    for j in range(i + 1, N):
        # Interaction term between qubit i and qubit j
        #print(i,j)
        hzz = qt.tensor([sz if k == i or k == j else I for k in range(N)]) # assume obc
        ising_ham += (V/N)*hzz
  
  #print(hz,hx,hzz)
  
  return(ising_ham)

In [72]:
omega = 1
N = 2
delta = 1
V = 3
hamiltonian=hamiltonian_ising(N,omega,delta,V)

In [73]:
hamiltonian

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[-0.5  1.   1.   0. ]
 [ 1.  -1.5  0.   1. ]
 [ 1.   0.  -1.5  1. ]
 [ 0.   1.   1.   3.5]]

In [75]:
sm = qt.operators.sigmam()
I = qt.operators.identity(2)
kappa = 1
c_ops = [qt.tensor([sm if i==j  else I for j in range(N)]) for i in range(N)]
c_ops = [np.sqrt(kappa) * c_op for c_op in c_ops]
#L = qt.liouvillian(hamiltonian, c_ops).full()
L_dag = qt.liouvillian(hamiltonian, c_ops).dag().full()

In [76]:
c_ops

[Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
 Qobj data =
 [[0. 0. 0. 0.]
  [0. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]],
 Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = False
 Qobj data =
 [[0. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 0.]
  [0. 0. 1. 0.]]]

In [82]:
# diagonalization using scipy eigsh
val,vec = sp.sparse.linalg.eigsh(L_dag,2,which='SM')
l2 = vec[:,1]

In [87]:
# evd using numpy
val,mat = np.linalg.eig(L_dag)
np.argsort(abs(val.real))[1]

In [123]:
# checking the eigen value equation l.H.S
val[10]*mat[:,10]

array([-1.69694508e-16-1.38386355e-16j,  5.74861396e-01+1.55039023e-01j,
       -5.74861396e-01-1.55039023e-01j,  3.04343329e-16-1.32933495e-15j,
        1.30276646e-01-2.89926915e-01j,  4.45548388e-02-7.49227037e-01j,
        2.58583595e-01-6.70934899e-01j,  3.63940487e-01-3.61194550e+00j,
       -1.30276646e-01+2.89926915e-01j, -2.58583595e-01+6.70934899e-01j,
       -4.45548388e-02+7.49227037e-01j, -3.63940487e-01+3.61194550e+00j,
       -3.55944949e-16-5.01657679e-16j,  2.06158283e-02+7.44285344e-03j,
       -2.06158283e-02-7.44285344e-03j, -2.13886771e-16-2.12915141e-16j])

In [124]:
# Checking the eigen value equation R.H.S
np.matmul(L_dag,mat[:,10])

array([-1.66533454e-16-5.89805982e-17j,  5.74861396e-01+1.55039023e-01j,
       -5.74861396e-01-1.55039023e-01j,  5.68989300e-16-1.36349265e-15j,
        1.30276646e-01-2.89926915e-01j,  4.45548388e-02-7.49227037e-01j,
        2.58583595e-01-6.70934899e-01j,  3.63940487e-01-3.61194550e+00j,
       -1.30276646e-01+2.89926915e-01j, -2.58583595e-01+6.70934899e-01j,
       -4.45548388e-02+7.49227037e-01j, -3.63940487e-01+3.61194550e+00j,
        3.66894015e-16+5.98479599e-17j,  2.06158283e-02+7.44285344e-03j,
       -2.06158283e-02-7.44285344e-03j,  1.67400815e-16-5.88938620e-16j])

In [149]:
# left eigen matrice of second lowest eigenvalue
l = mat[:,10]
l= np.matrix(l.reshape(4,4))

In [152]:
l[abs(l) < 1e-8] = 0

In [153]:
l

matrix([[ 0.        +0.00000000e+00j,  0.01781156-1.08297757e-01j,
         -0.01781156+1.08297757e-01j,  0.        +0.00000000e+00j],
        [-0.05558165-1.85355918e-02j, -0.1382356 +5.67409409e-03j,
         -0.12783165-3.50267557e-02j, -0.66917565+1.94289029e-16j],
        [ 0.05558165+1.85355918e-02j,  0.12783165+3.50267557e-02j,
          0.1382356 -5.67409409e-03j,  0.66917565+0.00000000e+00j],
        [ 0.        +0.00000000e+00j,  0.00098408-3.91859638e-03j,
         -0.00098408+3.91859638e-03j,  0.        +0.00000000e+00j]])

In [ ]:
# Solving master equation and state dissipation

psi0 = qt.tensor(qt.basis(2,0),qt.basis(2,1))
final_state = qt.steadystate(hamiltonian,c_ops)
#final_state.full() # density matrix corresponding to zero eigen value

tlist = omega*np.linspace(0, 10, 100)
medata = qt.mesolve(hamiltonian, psi0, tlist, c_ops, [])

In [ ]:
# calculation of Hilbert Distances

hilbert_distance = []

for state in medata.states:
  dis = qt.hilbert_dist(state,final_state)
  hilbert_distance.append(dis)

plt.plot(tlist,hilbert_distance)
plt.xlabel('time')
plt.ylabel('hilbert distance')
plt.show()